## 패키지 다운로드

In [2]:
!pip install -qU langchain langchain-community langchain-google-genai langchain-huggingface langchain-chroma pdfplumber

In [3]:
!pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               1.8.1
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.11.15
aiosignal                                1.3.2
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.1
altair                                   5.5.0
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                                    4.9.0
argon2-cffi                              25.1.0
argon2-cffi-bindings                     21.2.0
array_record                             0.7.2
arviz                                    0.21.0
astropy                             

## 구글 드라이브 연결

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

os.getcwd()

'/content'

In [6]:
os.listdir()

['.config', 'drive', 'sample_data']

In [7]:
%pwd

'/content'

In [8]:
%cd drive/MyDrive/250707/corpus

/content/drive/MyDrive/250707/corpus


## 모듈화 아닌 버전

### Langsmith 를 이용한 응답 추적

In [9]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["LANGCHAIN_TRACING"] = userdata.get('LANGSMITH_TRACING')
os.environ["LANGCHAIN_ENDPOINT"] = userdata.get('LANGSMITH_ENDPOINT')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGSMITH_PROJECT')

### 질의응답 시스템에 사용할 문서 읽기

In [10]:
from langchain_community.document_loaders import PDFPlumberLoader

docs = PDFPlumberLoader("거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf").load()

In [11]:
print(docs)

[Document(metadata={'source': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'file_path': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'page': 0, 'total_pages': 84, 'CreationDate': "D:20250106224410+09'00'", 'Creator': 'PDF 2022 13.0.0.564', 'ModDate': "D:20250106224430+09'00'", 'Producer': 'PDF 2022 13.0.0.564'}, page_content='공 학 석 사 학 위 논 문\n거대언어모델(LLM) 기반 질의응답\n시스템의 표 데이터 이해도를\n높이는 전처리 기법\n지 도 교 수 박 유 현\n공 동 지 도 교 수 이 정 훈\n2025년 2월\n동 의 대 학 교 대 학 원\n컴 퓨 터 소 프 트 웨 어 공 학 과\n정 민 수\n'), Document(metadata={'source': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'file_path': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'page': 1, 'total_pages': 84, 'CreationDate': "D:20250106224410+09'00'", 'Creator': 'PDF 2022 13.0.0.564', 'ModDate': "D:20250106224430+09'00'", 'Producer': 'PDF 2022 13.0.0.564'}, page_content='공 학 석 사 학 위 논 문\n거대언어모델(LLM) 기반 질의응답\n시스템의 표 데이터 이해도를\n높이는 전처리 기법\n지 도 교 수 박 유 현\n공 동 지 도 교 수 이 정 훈\n이 논문을 공학 석사학위논문으로 제출함\n2024년 12월\n동 의 대 학 교 대 

In [12]:
print(len(docs))

84


In [13]:
docs[0]

Document(metadata={'source': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'file_path': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'page': 0, 'total_pages': 84, 'CreationDate': "D:20250106224410+09'00'", 'Creator': 'PDF 2022 13.0.0.564', 'ModDate': "D:20250106224430+09'00'", 'Producer': 'PDF 2022 13.0.0.564'}, page_content='공 학 석 사 학 위 논 문\n거대언어모델(LLM) 기반 질의응답\n시스템의 표 데이터 이해도를\n높이는 전처리 기법\n지 도 교 수 박 유 현\n공 동 지 도 교 수 이 정 훈\n2025년 2월\n동 의 대 학 교 대 학 원\n컴 퓨 터 소 프 트 웨 어 공 학 과\n정 민 수\n')

In [14]:
print(type(docs))

<class 'list'>


### 문서 분할

#### 한국어 형태소 단위

In [15]:
!python3 -m pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 40.6 MB/s eta 0:00:00


In [16]:
from konlpy.tag import Kkma, Okt

okt = Okt()
kkma = Kkma()

In [17]:
def len_okt(text):
    tokens = [token for token in okt.morphs(text)]

    return len(tokens)

In [18]:
def okt_tokenize(text):
    return [token for token in okt.morphs(text)]

In [19]:
def len_kkma(text):
    tokens = [token for token in kkma.morphs(text)]

    return len(tokens)

In [20]:
def kkma_tokenize(text):

    return [token for token in kkma.morphs(text)]

#### 분할기 생성

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len_okt
)

In [22]:
texts = text_splitter.split_documents(docs)


In [23]:
texts[0]

Document(metadata={'source': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'file_path': '거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법.pdf', 'page': 0, 'total_pages': 84, 'CreationDate': "D:20250106224410+09'00'", 'Creator': 'PDF 2022 13.0.0.564', 'ModDate': "D:20250106224430+09'00'", 'Producer': 'PDF 2022 13.0.0.564'}, page_content='공 학 석 사 학 위 논 문\n거대언어모델(LLM) 기반 질의응답\n시스템의 표 데이터 이해도를\n높이는 전처리 기법\n지 도 교 수 박 유 현\n공 동 지 도 교 수 이 정 훈\n2025년 2월\n동 의 대 학 교 대 학 원\n컴 퓨 터 소 프 트 웨 어 공 학 과\n정 민 수')

In [24]:
print(len(docs))

84


In [25]:
print(len(texts))

83


### 임베딩 모델 생성

#### 허깅페이스 임베딩 모델

In [26]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "nlpai-lab/KURE-v1"  # 한국어 모델
# model_kwargs = {'device': 'cpu'}  # cpu 사용
model_kwargs = {'device': 'cuda'}  # gpu 사용
encode_kwargs = {'normalize_embeddings': True}
hf_embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [27]:
hf_embedding_test = hf_embedding_model.embed_query("안녕하세요.")

hf_embedding_test

[-0.024604560807347298,
 0.025116274133324623,
 -0.03335217386484146,
 -0.00202204124070704,
 -0.014855310320854187,
 -0.05262487009167671,
 -0.01889931783080101,
 -0.06538813561201096,
 0.024237262085080147,
 0.0011860248632729053,
 0.00400763563811779,
 0.0315011665225029,
 0.016683271154761314,
 -0.014650936238467693,
 0.02357589080929756,
 -0.033598393201828,
 0.013595260679721832,
 -0.03999217972159386,
 0.00567800784483552,
 -0.05042201653122902,
 -0.03937114030122757,
 0.002800587099045515,
 -0.008931447751820087,
 -0.014998788945376873,
 0.007617462892085314,
 0.04348299279808998,
 -0.021915579214692116,
 0.021404793485999107,
 0.01784462481737137,
 -0.03360377252101898,
 0.027148298919200897,
 0.04266786202788353,
 -0.004230374936014414,
 -0.03646565601229668,
 -0.006372441072016954,
 -0.039430443197488785,
 -0.015058347955346107,
 -0.010653108358383179,
 -0.05758838355541229,
 0.04578414559364319,
 0.031237639486789703,
 0.014935863204300404,
 0.003296470735222101,
 -0.059383

In [28]:
print(len(hf_embedding_test))

1024


### 문서 임베딩

In [29]:
%pwd

'/content/drive/MyDrive/250707/corpus'

In [30]:
%cd ..

/content/drive/MyDrive/250707


In [31]:
save_directory = "./chroma_docs_db"

In [32]:
from langchain_chroma import Chroma
import os
import shutil

print("\n잠시만 기다려주세요.\n\n")

# 벡터저장소가 이미 존재하는지 확인
if os.path.exists(save_directory):
    shutil.rmtree(save_directory)
    print(f"디렉토리 {save_directory}가 삭제되었습니다.\n")

print("문서 벡터화를 시작합니다. ")
db = Chroma.from_documents(docs, hf_embedding_model, persist_directory=save_directory)  # HuggingFace Embedding Model
# db = Chroma.from_documents(docs, oepnai_embedding_model, persist_directory=save_directory)  # OpenAI Embedding Model
print("새로운 Chroma 데이터베이스가 생성되었습니다.\n")


잠시만 기다려주세요.


문서 벡터화를 시작합니다. 
새로운 Chroma 데이터베이스가 생성되었습니다.



In [33]:
retriever = db.as_retriever(
    search_kwargs={"k": 3},
)

### 채팅에 사용될 거대언어모델 생성

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Answer in Korean.

#Context:
{context}
""",
        ),
        ("human", "{question}"),
    ]
)

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    disable_streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [36]:
def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

chain = {
    # "context": retriever,
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
} | prompt | llm | StrOutputParser()

In [38]:
question = "본 논문에서 제안하는 전처리 기법은 뭐가 있는지 알려줘."

response = chain.invoke(question)

In [39]:
print(response)

본 논문에서 제안하는 전처리 기법은 다음과 같이 두 가지입니다:

1.  **구분자 기반 전처리 기법**: '/'를 사용하여 표 속성을 구분하고, 하위 속성은 중괄호 또는 대괄호로 묶어 표 구조를 자연어로 변환하는 기법입니다.
2.  **한국어 조사 체계 기반 전처리 기법**: 한국어의 조사 체계(예: "~에서", "~은", "~의", "~이다")를 활용하여 표의 열(Row) 값을 완전한 자연어 문장으로 변환하는 기법입니다.


In [40]:
question = "본 논문 제목을 알려줘."

response = chain.invoke(question)

In [41]:
print(response)

본 논문의 제목은 "거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법"입니다.


### 하이브리드 검색기 사용

In [42]:
%pip install rank_bm25

In [43]:
from langchain_community.retrievers import BM25Retriever

bm_retriever = BM25Retriever.from_documents(
    documents=docs,
    preprocess_func=okt_tokenize,
)

bm_retriever.k = 3

In [44]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm_retriever],
    weights=[0.5, 0.5],
)

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

chain = {
    # "context": retriever | RunnableLambda(format_docs),
    "context": ensemble_retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
} | prompt | llm | StrOutputParser()

In [46]:
question = "본 논문에서 제안하는 전처리 기법은 뭐가 있는지 알려줘."

response = chain.invoke(question)

In [47]:
print(response)

본 논문에서 제안하는 전처리 기법은 두 가지입니다.

1.  **구분자 기반 전처리 기법**: '/'를 사용하여 표 속성을 구분하고, 하위 속성은 중괄호 또는 대괄호로 묶어 표 구조를 자연어로 변환하는 기법입니다.
2.  **한국어 조사 체계 기반 전처리 기법**: 한국어의 조사 체계를 활용하여 표의 열(Row) 값을 하나의 완전한 자연어 문장으로 변환하는 기법입니다. 구체적으로는 표 제목에 "~에서", 표 속성에 "~은", 표 속성 깊이를 표현하기 위해 "~의", 그리고 셀 값에 "~이다"와 같은 조사를 사용합니다.


In [48]:
question = "본 논문 제목을 알려줘."

response = chain.invoke(question)

In [49]:
print(response)

본 논문의 제목은 "거대언어모델(LLM) 기반 질의응답 시스템의 표 데이터 이해도를 높이는 전처리 기법" 입니다.
